# Appendix 3 - Raspberry-based acquisition

This appendix describes the process for implement an acquisition server based on Raspberry Pi, the main goal of using Raspberry is to get a fully controlled system. The operating system chosen for this purpose is [Arch Linux ARM](https://archlinuxarm.org/).

## Step 1 - Install Arch Linux ARM

The process of installation is very simple, there is a [guide](https://archlinuxarm.org/platforms/armv8/broadcom/raspberry-pi-4) on the official site.  
First at all run a complete system update and install and [configure sudo](https://wiki.archlinux.org/index.php/sudo) to use it with the default user.

In [ ]:
$ pacman -Suy
$ pacman -S sudo

## TL;WR
Steps 2 to 6 can be done by a single command, that's not fun, but:

In [ ]:
$ pacman -S wget
$ wget https://raw.githubusercontent.com/UN-GCPDS/openbci-stream/master/cmd/stream_tlwr.sh
$ chmod a+x stream_tlwr.sh
$ ./stream_tlwr.sh

And jump to [Step 7](#Step-7---Test-server)

## Step 2 - Install dependencies

The basic dependencies like `python-pip`, `gcc`, `cython` and `hdf5` can be installed with:

In [ ]:
$ pacman -S python python-pip gcc cython hdf5

The main module `openbci-stream` can be installed with:

In [ ]:
$ pip install openbci-stream

`openbci-stream` will install a set of scripts for easy environment configuration.

The following command will download the [PKGBUILD](https://github.com/UN-GCPDS/archlinux-PKGBUILD) for [kafka](https://kafka.apache.org/) and [zookeeper](https://zookeeper.apache.org/) and then install them:

In [ ]:
$ stream_install_kafka.sh 

<div class="alert alert-info"> <b>Note</b><br>

Arch Linux ARM could be a bit fussy about install `scipy` from `pip`, the best option here is use the default package manager:  
`pacman -S python-scipy --overwrite='/usr/lib/python3.8/site-packages/*' --overwrite='/usr/bin/*'`  

</div>

## Step 3 - Configure Kafka

The kafka configuration can be done simply by executing the script `stream_configure_kafka.sh` included on `openbci-stream` package:

In [ ]:
$ stream_configure_kafka.sh 

## Step 4 - Configure an access point

The main purpose of this guide is to configure the Raspberry as an access point, the next script will configure a WiFi network called `OpenBCI-Stream` with password `raspberrypi`:

In [ ]:
$ stream_access_point.sh

<div class="alert alert-info"> <b>Note</b><br>
    
If you are connected via WiFi, this connection will be lost after restart the Raspberry, you may wish to connect using other methods (Ethernet, USB) in ways to continue the configuration.  

</div>

## Step 5 - Configure time server

This step is very important if you want to stream markers with high time precision, consist in configure the Raspberry as an [NTP](https://wiki.archlinux.org/index.php/Network_Time_Protocol_daemon#NTP_server_mode) server.

In [ ]:
$ stream_configure_ntpd.sh

The command above install dependencies, configure the server, and start the daemons.  
Now in our client, assuming that we are using a GNU/Linux, we must add `server 192.168.1.1 iburst prefer` on the configuration file `/etc/ntp.conf` and restart `ntpd`

## Step 6 - Start acquisition daemons

In [ ]:
$ stream_eeg systemd
$ stream_rpyc systemd

The commands above will create the respective services for acquisition streaming and distributed access. And then, for initialize them:

In [ ]:
$ systemctl enable stream_eeg stream_rpyc
$ systemctl start stream_eeg stream_rpyc

## Step 7 - Test server

<div class="alert alert-warning"> <b>Warning</b><br>
    
From this step forward the commands <b>are not</b> executed on Raspberry, but on your <b>main computer</b>. 

</div>

For a complete test, we will need an OpenBCI board connected, in any mode, Serial or WiFi (so be sure about configuring serial port permissions).

### WiFi Shield
For a WiFi shield, we must know the correct IP, usually this IP not change over time, so if we found it the first time is very likely that we can use the same in other opportunities.  
You can use [nmap](https://wiki.archlinux.org/index.php/nmap#Ping_scan) to scan devices:

In [ ]:
$ ssh alarm@192.168.1.1
$ nmap -sn 192.168.1.2/24

### Test boards
At this point we have `OpenBCI-Stream` installed on Raspberry, now we need to install `OpenBCI-Stream` on the main computer too, also [Kafka must be installed and running](02-kafka_configuration.ipynb).

In [ ]:
$ openbci_cli wifi --ip 192.168.1.113 --host 192.168.1.1 -c SAMPLE_RATE_250SPS

Here, `192.168.1.113` is the IP assigned from Raspberry to the WiFi module and `192.168.1.1` is the Raspberry IP itself. If everything is ok, a data stream will be printed on the terminal.

### Test NTP server

We can test and synchronize with the server with this command:

In [10]:
ntpdate -d 192.168.1.1 

26 Oct 17:38:28 ntpdate[32205]: ntpdate 4.2.8p15@1.3728-o Wed Jul  1 17:02:17 UTC 2020 (1)
Looking for host 192.168.1.1 and service ntp
192.168.1.1 reversed to _gateway
host found : _gateway
transmit(192.168.1.1)
receive(192.168.1.1)
transmit(192.168.1.1)
receive(192.168.1.1)
transmit(192.168.1.1)
receive(192.168.1.1)
transmit(192.168.1.1)
receive(192.168.1.1)

server 192.168.1.1, port 123
stratum 4, precision -20, leap 00, trust 000
refid [162.159.200.1], root delay 0.139526, root dispersion 0.078171
reference time:      e341cce9.4ef3aa7d  Mon, Oct 26 2020 17:32:09.308
originate timestamp: e341ce6b.10600a1b  Mon, Oct 26 2020 17:38:35.063
transmit timestamp:  e341ce6b.109e15f3  Mon, Oct 26 2020 17:38:35.064
filter delay:  0.48293    0.06233    0.04648    0.06221   
               ----       ----       ----       ----      
filter offset: +0.188384  -0.019893  -0.027461  -0.019337 
               ----       ----       ----       ----      
delay 0.04648, dispersion 0.03293, offset -0.02

In case of obtaining this output `no server suitable for synchronization found`, is due that the server (Raspberry) [has not synchronized itself with other servers](http://www.markelov.org/wiki/index.php?title=NTP._Server_dropped:_strata_too_high), just wait and try again.

Is possible to check wich server is used for syncronization with the follow command, the syncronization not ocurss instantanely, in some cases is necesario wait up to 10 minutes.

In [8]:
!ntpq -pn

     remote           refid      st t when poll reach   delay   offset  jitter
*192.168.1.1     162.159.200.1    4 u   14   64    1    2.017  -33.339   1.614


### Troubleshooting

  * **No streaming data:** Make sure that the WiFi module was still connected and the Serial devices have read and write permissions.
  * **Kafka without connection:** Kafka must be running on every single machine that wants to connect to the stream, there is no special configuration, the default is enough.
 